In [1]:
#800 node vs 400
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
import tqdm
tf.disable_eager_execution()
%matplotlib inline
from tensorflow.keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply, Reshape
from tensorflow.keras.layers import RepeatVector, Dense, Activation, Lambda, Embedding,Dropout,Add,Softmax,GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model, Model
from tensorflow.keras import backend as K

In [2]:
tf.__version__

'1.15.0'

In [3]:
loadmodel=0
forceteach=0
orignial=0
pretrain_fr=1
attention='Loung' #'Bahdanau'
layerofencoderlstm=1

In [4]:
##NEEDS DROP OUT


#french pretrained embedding
#encoderstate to decoder 
#forceteach
#Bahdanau 
#attention='Bahdanau' #loungAttention
unkcntlimit=1
uniquewordscnt=4000

#tanh vs none
#vocabRatio=0.89


rnn_size=400
activationF='tanh'
loungAttentionNodeCnt=rnn_size
lstmregulizer=None
denseregulizer=None

wordvectorsize=100

maxlength=30
lstmdropout=0.1
recur_dropout=0.1
betweenlayerdropout=0.5
convertoworddropout=0.5

decoderlstmdropout=0.1
decoderrecurdropout=0.1

In [5]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [6]:
# English source data
if (orignial==1):
    with open("./data/small_vocab_en", "r", encoding="utf-8") as f:
        source_text = f.read()

    # French target data
    with open("./data/small_vocab_fr", "r", encoding="utf-8") as f:
        target_text = f.read()
else:
    
    with open("../data/small_vocab_en_UN", "r", encoding="utf-8") as f:
        source_text = f.read()

    # French target data
    with open("../data/small_vocab_fr_UN", "r", encoding="utf-8") as f:
        target_text = f.read()

In [7]:

newstring=[]
skipmod=0
for i in range(len(source_text)*2):
    try:
        char=source_text[i]
    except:
        print(i)
        break
    if(char==' ' or char=="\\" ):
        skipmod=0
    if(skipmod==1):
        continue

    if(char.isnumeric()==True):
        continue
    elif(char==')'):
        continue
    elif(char=='('):
        continue
    elif(char=='"'):
        continue
    elif(char=='['):
        continue
    elif(char==']'):
        continue
    elif(char==';'):
        continue
    elif(char=='-'):
        newstring.append(' ')
        continue        
    elif(char==':'):
        newstring.append(' ')
        newstring.append(':')
        newstring.append(' ')
        continue    

  #  elif(char=='.' and source_text[i-1].isnumeric()==True):
  #      continue
    
    elif(char==',' and source_text[i-1]!=' '):
        newstring.append(' ')
        newstring.append(',')
     
    elif(char=='.' and source_text[i-1]!=' '):
        newstring.append(' ')
        newstring.append('.')
        
    else:
        newstring.append(char)
    
newstring=''.join(newstring)    
del source_text
source_text=newstring
       
newstring2=[]
skipmod=0
for i in range(len(target_text)*2):
    try:
        char=target_text[i]
    except:
        print(i)
        break
    if(char==' ' or '\\'):
        skipmod=0
    if(skipmod==1):
        continue

    if(char.isnumeric()==True):
        continue
    elif(char==')'):
        continue
    elif(char=='('):
        continue
    elif(char=='"'):
        continue
    elif(char=='['):
        continue
    elif(char==']'):
        continue
    elif(char==';'):
        continue
    elif(char=='-'):
        newstring2.append(' ')
    elif(char==':'):
        newstring2.append(' ')
        newstring2.append(':')
        newstring2.append(' ')
        continue    

   # elif(char=='.' and target_text[i-1].isnumeric()==True):
   #     continue
    
    elif(char==',' and target_text[i-1]!=' '):
        newstring2.append(' ')
        newstring2.append(',')
      
    elif(char=='.' and target_text[i-1]!=' '):
        newstring2.append(' ')
        newstring2.append('.')
      
    else:
        newstring2.append(char)
    
newstring2=''.join(newstring2)   
del target_text
target_text=newstring2

view_sentence_range = (0, 20)


print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word.lower(): None for word in source_text.split()})))


print("-"*5 + "English Text" + "-"*5)
sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))
print('Max number of words in a sentence: {}'.format(np.max(word_counts)))

print()
print("-"*5 + "French Text" + "-"*5)
sentences = target_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))
print('Max number of words in a sentence: {}'.format(np.max(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

123279247
136885361
Dataset Stats
Roughly the number of unique words: 76939
-----English Text-----
Number of sentences: 800001
Average number of words in a sentence: 25.53115308605864
Max number of words in a sentence: 1449

-----French Text-----
Number of sentences: 800001
Average number of words in a sentence: 27.901697622877972
Max number of words in a sentence: 1607

English sentences 0 to 20:
UNITED NATIONS
E
Economic and Social Council
Distr .
GENERAL
 February 
Original :  FRENCH
ECONOMIC COMMISSION FOR EUROPE
INLAND TRANSPORT COMMITTEE
Working Party on the Construction of Vehicles
One hundred and seventeenth session ,
  March  , agenda item  .
PROPOSAL FOR DRAFT SUPPLEMENT  TO REGULATION No . 
Vehicle alarm systems
Addendum 
Transmitted by the expert from the International Organization of
Motor Vehicle Manufacturers OICA
Note :  The text reproduced below was prepared by the expert from OICA in accordance with the decision of GRSG at its seventy fifth session TRANS/WP ./GRSG/ .


In [ ]:
source_textlist=source_text.splitlines(keepends=True)
target_textlist=target_text.splitlines(keepends=True)
#delete long lines and lines with too many words not alphbet
i=-1

while(i<len(source_textlist)):
    i+=1
    if(i%1000==0):
        print(i)
    try:
        line=source_textlist[i]
    except:
        print(i)
        break  
    wordcnt=0
    notalphbetcnt=0
    line=line.split()
    for word in line:
        wordcnt+=1
        if(word.isalpha()== False):
            notalphbetcnt+=1
    
    if(wordcnt>29):
        
        del source_textlist[i]
        del target_textlist[i]
        i-=1
    elif(wordcnt<3):
        
        del source_textlist[i]
        del target_textlist[i]   
        i-=1    
    elif(notalphbetcnt/wordcnt>0.3):
        
        del source_textlist[i]
        del target_textlist[i]   
        i-=1    
i=-1
while(i<len(target_textlist)):
    i+=1
    if(i%1000==0):
        print(i)
    try:
        line=target_textlist[i]
    except:
        print(i)
        break  
    wordcnt=0
    notalphbetcnt=0
    line=line.split()
    for word in line:
        wordcnt+=1

    
    if(wordcnt>29):
        
        del source_textlist[i]
        del target_textlist[i]
        i-=1


print(len(source_textlist))
print(len(target_textlist))

source_text=''.join(source_textlist)
target_text=''.join(target_textlist)
del source_textlist
del target_textlist

In [9]:
random_index = 30000

print("-"*5 + "English example" + "-"*5)
print(source_text.split("\n")[random_index])


print()
print("-"*5 + "French example" + "-"*5)
print(target_text.split("\n")[random_index])


print("-"*5 + "English example" + "-"*5)
print(source_text.split("\n")[random_index+5])


print()
print("-"*5 + "French example" + "-"*5)
print(target_text.split("\n")[random_index+5])

-----English example-----
a . Access to financial resources for sustainable development

-----French example-----
a Accès aux ressources financières en vue du développement durable
-----English example-----
v Reports submitted by other intergovernmental organizations , for example , the Commonwealth Secretariat and the South Pacific Regional Environment Programme

-----French example-----
v Rapports présentés par d'autres organisations intergouvernementales , par exemple le Secrétariat du Commonwealth et le Programme régional pour l'environnement du Pacifique Sud


In [10]:
view_sentence_range = (0, 20)


print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word.lower(): None for word in source_text.split()})))


print("-"*5 + "English Text" + "-"*5)
sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))
print('Max number of words in a sentence: {}'.format(np.max(word_counts)))

print()
print("-"*5 + "French Text" + "-"*5)
sentences = target_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))
print('Max number of words in a sentence: {}'.format(np.max(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 45980
-----English Text-----
Number of sentences: 370833
Average number of words in a sentence: 14.75138943945118
Max number of words in a sentence: 29

-----French Text-----
Number of sentences: 370833
Average number of words in a sentence: 15.788228124249999
Max number of words in a sentence: 29

English sentences 0 to 20:
Economic and Social Council
ECONOMIC COMMISSION FOR EUROPE
INLAND TRANSPORT COMMITTEE
Working Party on the Construction of Vehicles
One hundred and seventeenth session ,
PROPOSAL FOR DRAFT SUPPLEMENT  TO REGULATION No . 
Vehicle alarm systems
Transmitted by the expert from the International Organization of
Motor Vehicle Manufacturers OICA
Note :  The text reproduced below was prepared by the expert from OICA in accordance with the decision of GRSG at its seventy fifth session TRANS/WP ./GRSG/ .
GE .  E page
Title of document , amend to read : 
“PROPOSAL FOR THE  SERIES OF AMENDMENTS TO REGULATION No . ”
“ . . . an a

In [12]:


vocabRatio=0.79
from collections import Counter 

source_text_forvaca = [item for items, c in Counter(source_text.lower().split()).most_common() 
                                      for item in [items] * c] 
source_text_forvacac = [c for items, c in Counter(source_text.lower().split()).most_common() 
                                      for item in [items] * c] 
print(source_text_forvaca[int(len(source_text_forvaca)*vocabRatio)])
print(source_text_forvacac[int(len(source_text_forvaca)*vocabRatio)])


source_vocab=[]
while(len(source_vocab)<uniquewordscnt and vocabRatio<1.0):
    vocabRatio+=0.005
    print(vocabRatio)
    source_text_forvacaloop=source_text_forvaca[:int(len(source_text_forvaca)*vocabRatio)]
    source_vocab = list(set(source_text_forvacaloop))

print('vocabRatio',vocabRatio)
# 构造法语词典
if(vocabRatio<1):
    vocabRatio=vocabRatio-0.01
target_vocab=[]
target_text_forvaca = [item for items, c in Counter(target_text.lower().split()).most_common() 
                                      for item in [items] * c] 
target_text_forvacac = [c for items, c in Counter(target_text.lower().split()).most_common() 
                                      for item in [items] * c] 
print(target_text_forvaca[int(len(target_text_forvaca)*vocabRatio)-1])
print(target_text_forvacac[int(len(target_text_forvaca)*vocabRatio)-1])
target_text_forvaca=target_text_forvaca[:int(len(target_text_forvaca)*vocabRatio)]
target_vocab = list(set(target_text_forvaca))



source_vocab=source_vocab[:20000]
target_vocab=target_vocab[:30000]
print("The size of English vocab is : {}".format(len(source_vocab)))
print("The size of French vocab is : {}".format(len(target_vocab)))

haiti
740
0.795
0.8
0.805
0.81
0.8150000000000001
0.8200000000000001
0.8250000000000001
0.8300000000000001
0.8350000000000001
0.8400000000000001
0.8450000000000001
0.8500000000000001
0.8550000000000001
0.8600000000000001
0.8650000000000001
0.8700000000000001
0.8750000000000001
0.8800000000000001
0.8850000000000001
0.8900000000000001
0.8950000000000001
0.9000000000000001
0.9050000000000001
0.9100000000000001
0.9150000000000001
0.9200000000000002
0.9250000000000002
0.9300000000000002
0.9350000000000002
0.9400000000000002
vocabRatio 0.9400000000000002
s'être
75
The size of English vocab is : 4010
The size of French vocab is : 5298


In [13]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
source_textlist=source_text.splitlines(keepends=True)
target_textlist=target_text.splitlines(keepends=True)

i=-1

while(i<len(source_textlist)):
    i+=1
 
    if(i%1000==0):
        print(i)
    try:
        line=source_textlist[i]
    except:
        print(i)
        break  
    wordcnt=0
    unkcnt=0
   
    line=line.split()
    for word in line:
        wordcnt+=1
        if(word.lower() not in source_vocab ):
       
            unkcnt+=1
    
    
    if(unkcnt>unkcntlimit or unkcnt/wordcnt>0.2):
      
      
        del source_textlist[i]
        del target_textlist[i]
        i-=1


i=-1
while(i<len(target_textlist)):
    i+=1
    if(i%1000==0):
        print(i)
    try:
        line=target_textlist[i]
    except:
        print(i)
        break  
    wordcnt=0
    unkcnt=0
   
    line=line.split()
    for word in line:
        wordcnt+=1
        if(word.lower() not in target_vocab ):

            unkcnt+=1
            
    
    
    if(unkcnt>unkcntlimit or unkcnt/wordcnt>0.2):

        del source_textlist[i]
        del target_textlist[i]
        i-=1


print(len(source_textlist))
print(len(target_textlist))


source_text=''.join(source_textlist)
target_text=''.join(target_textlist)
del source_textlist
del target_textlist

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
23000
23000
24000
25000
26000
27000
28000
28000
28000
29000
30000
31000
32000
33000
34000
35000
36000
36000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
48000
49000
50000
50000
50000
51000
51000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
62000
63000
63000
63000
63000
64000
64000
64000
65000
66000
66000
66000
67000
67000
68000
69000
70000
70000
70000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
80000
80000
80000
81000
82000
83000
84000
85000
86000
86000
87000
88000
89000
90000
91000
91000
92000
93000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
105000
106000
107000
108000
109000
110000
110000
110000
111000
112000
113000
114000
115000
116000
117000
118000
118000
118000
119000
119000
120000
120000
121000
122000
122000
123000
124000
124000
1

In [15]:




source_vocab=source_vocab[:20000]
target_vocab=target_vocab[:30000]
print("The size of English vocab is : {}".format(len(source_vocab)))
print("The size of French vocab is : {}".format(len(target_vocab)))

The size of English vocab is : 4010
The size of French vocab is : 5298


In [16]:
print("The size of English vocab is : {}".format(len(source_vocab)))
print("The size of French vocab is : {}".format(len(target_vocab)))

The size of English vocab is : 4010
The size of French vocab is : 5298


In [17]:
# 特殊字符
SOURCE_CODES = ['<PAD>', '<UNK>']
TARGET_CODES = ['<PAD>', '<EOS>', '<UNK>', '<GO>']  # 在target中，需要增加<GO>与<EOS>特殊字符

In [18]:
# 构造英文映射字典
source_vocab_to_int = {word: idx for idx, word in enumerate(SOURCE_CODES + source_vocab)}

source_int_to_vocab = {idx: word for idx, word in enumerate(SOURCE_CODES + source_vocab)}


    
    
# 构造法语映射词典
target_vocab_to_int = {word: idx for idx, word in enumerate(TARGET_CODES + target_vocab)}
target_int_to_vocab = {idx: word for idx, word in enumerate(TARGET_CODES + target_vocab)}

In [19]:
print("The size of English Map is : {}".format(len(source_vocab_to_int)))
print("The size of French Map is : {}".format(len(target_vocab_to_int)))

The size of English Map is : 4012
The size of French Map is : 5302


In [20]:
print(target_int_to_vocab[100])

essentiellement


In [21]:
def checksentence(sentence):
    invalid=0
    total=1
   # for word in sentence.lower().split():
    #    if(word.isalpha()==False and word !=',' and word !='.'):
    #        invalid+=1
    #    total+=1
    #if(total<3):
    #    return False
   # if(invalid/total>0.2):

   #     return False
    return True
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

def text_to_int(sentence, map_dict, max_length=maxlength, is_target=False):

    
    
    text_to_idx = []
    # unk index
    unk_idx = map_dict.get("<UNK>")
    pad_idx = map_dict.get("<PAD>")
    eos_idx = map_dict.get("<EOS>")
    
  
    
    for word in sentence.lower().split():
        if(hasNumbers(word)):
            text_to_idx.append(unk_idx)
        else:
            text_to_idx.append(map_dict.get(word, unk_idx))
    if is_target:
        text_to_idx.append(eos_idx)
    


    

    if len(text_to_idx) > max_length:
        return text_to_idx[:max_length]

    else:
        text_to_idx = text_to_idx + [pad_idx] * (max_length - len(text_to_idx))
        return text_to_idx

In [22]:

source_text_to_int = []
target_text_to_int = []
sourcelist=source_text.split("\n")
targetlist=target_text.split("\n")
index=0
for index in tqdm.tqdm(range(len(sourcelist))):
    sentence=sourcelist[index]
    if(checksentence(sentence)):
        source_text_to_int.append(text_to_int(sentence, source_vocab_to_int, maxlength, 
                                          is_target=False))
        target_text_to_int.append(text_to_int(targetlist[index], target_vocab_to_int, maxlength, 
                                              is_target=True))
    else:
        print(sentence)
    

100%|██████████| 233705/233705 [00:05<00:00, 45315.12it/s]


In [23]:
random_index = 20000

print("-"*5 + "English example" + "-"*5)
print(source_text.split("\n")[random_index])
print(source_text_to_int[random_index])

print()
print("-"*5 + "French example" + "-"*5)
print(target_text.split("\n")[random_index])
print(target_text_to_int[random_index])

print("-"*5 + "English example" + "-"*5)
print(source_text.split("\n")[random_index+1])
print(source_text_to_int[random_index+1])

print()
print("-"*5 + "French example" + "-"*5)
print(target_text.split("\n")[random_index+1])
print(target_text_to_int[random_index+1])

-----English example-----
A . The overall policy environment     
[3567, 3138, 1525, 1187, 76, 1799, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

-----French example-----
A . Contexte général des politiques de la dette     
[4663, 4110, 432, 1126, 4225, 4056, 3125, 2402, 4582, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
-----English example-----
B . A focus on the Paris Club     
[458, 3138, 3567, 530, 2557, 1525, 1672, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

-----French example-----
B . Le Club de Paris     
[572, 4110, 2976, 2, 3125, 2234, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [24]:
X = np.array(source_text_to_int)
Y = np.array(target_text_to_int)
print(X.shape)
print(Y.shape)

(233705, 30)
(233705, 30)


In [25]:
def transpose2(tensor,perm):
    x=tf.transpose(tensor, perm=perm)
    return x

In [26]:
from tensorflow.keras.layers import RepeatVector, Dense, Activation, Lambda, Embedding

with open("../data/enwiki_20180420_100d.txt", 'r') as f:
    words = set()
    word_to_vec_map = {}
    iii=0
    for line in f:
        iii+=1
      
        if(iii==1):
            continue
        line = line.strip().split()
        curr_word = line[0]
        if(curr_word[:5]=='ENTITY'):
            continue

        try:       
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
            words.add(curr_word)
        except:
            continue       
        

In [27]:


        
        
def pretrained_embedding_layer(word_to_vec_map, source_vocab_to_int):

    
    vocab_len = len(source_vocab_to_int)      
    emb_dim = word_to_vec_map["the"].shape[0]


    emb_matrix = np.zeros((vocab_len, emb_dim))
    
 
    for word, index in source_vocab_to_int.items():
        word_vector = word_to_vec_map.get(word, np.zeros(emb_dim))
        emb_matrix[index, :] = word_vector

    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)

    # build
    embedding_layer.build((None,))

    # set weights
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer
embedding_layer = pretrained_embedding_layer(word_to_vec_map, source_vocab_to_int)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [28]:
from tensorflow.keras.layers import RepeatVector, Dense, Activation, Lambda, Embedding

with open("../data/frwiki_20180420_100d.txt", 'r') as fr:
#with open("../data/glove.42B.300d.txt", 'r') as f:
    words = set()
    word_to_vec_mapfr = {}
    iii=0
    for line in fr:
        iii+=1
        if(iii==1):
            continue
        line = line.strip().split()
        curr_word = line[0]
        if(curr_word[:5]=='ENTITY'):
            continue
        #print(line[1])


        try:


       
            word_to_vec_mapfr[curr_word] = np.array(line[1:], dtype=np.float64)
            words.add(curr_word)
        except:
            continue
print(word_to_vec_mapfr['de'])
print(len(word_to_vec_mapfr))
print(type(target_vocab_to_int))
print(type(target_text_to_int))


[-0.0861 -0.1222 -0.0619 -0.3368  0.115  -0.2224  0.4958  0.1807 -0.1449
  0.1074  0.1161 -0.0583 -0.0113 -0.4103 -0.0344  0.4085  0.1163  0.3438
 -0.0576 -0.0821 -0.1362 -0.1616  0.1844  0.182   0.1505 -0.1365  0.2082
  0.2805 -0.1366  0.1316  0.0174 -0.0438  0.2987  0.03    0.3864  0.1577
  0.0414 -0.1326  0.0391  0.2384  0.2411 -0.0183 -0.0548 -0.2325 -0.2505
  0.0733  0.1344 -0.0767 -0.1943 -0.2422  0.1456  0.1247 -0.3724 -0.3578
  0.0514 -0.1402  0.102  -0.2958 -0.0759 -0.0165 -0.1523  0.1517  0.2418
 -0.3599  0.6086 -0.3629 -0.0387 -0.0588  0.1927  0.2059 -0.0409  0.2896
 -0.0852  0.1295  0.2394 -0.0134 -0.2348 -0.0374  0.0492 -0.3214  0.1219
 -0.0497 -0.252  -0.1881 -0.424  -0.5537 -0.1215 -0.2991  0.2913  0.0731
 -0.2589 -0.0038  0.4187 -0.1844 -0.2661 -0.1089  0.2849 -0.1919  0.1371
 -0.2906]
2500667
<class 'dict'>
<class 'list'>


In [29]:
if(pretrain_fr==1):
    def pretrained_embedding_layerfr(word_to_vec_mapfr, target_vocab_to_int):
        """
        构造Embedding层并加载预训练好的词向量（这里我使用的是100维）

        @param word_to_vec_map: 单词到向量的映射
        @param word_to_index: 单词到数字编码的映射
        """

        vocab_len = len(target_vocab_to_int)    
        emb_dim = word_to_vec_mapfr["de"].shape[0]

      
        emb_matrix = np.zeros((vocab_len, emb_dim))


        for word, index in target_vocab_to_int.items():
            word_vector = word_to_vec_mapfr.get(word, np.zeros(emb_dim))
            emb_matrix[index, :] = word_vector


        embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)

        # build
        embedding_layer.build((None,))

        # set weights
        embedding_layer.set_weights([emb_matrix])

        return embedding_layer
    embedding_layerfr = pretrained_embedding_layerfr(word_to_vec_mapfr, target_vocab_to_int)

In [30]:
if(pretrain_fr==0):
    class EmbeddingFrTrain(tf.keras.layers.Layer):
        def __init__(self, vocabcnt, wordvectorlength):
            super(EmbeddingFrTrain, self).__init__()

            self.vocabcnt=vocabcnt
            self.wordvectorlength=wordvectorlength
            #self.lstm_cell = LSTM(rnn_size,return_state=True,dropout=decoderlstmdropout,recurrent_dropout=decoderrecurdropout,return_sequences=False,name='decoder_cell')


        def get_config(self):

            config = super().get_config().copy()
            config.update({

                'vocabcnt':self.vocabcnt,
                'wordvectorlength':self.wordvectorlength,

            })
            return config

        def call(self, inputx):

            out=Embedding(self.vocabcnt, self.wordvectorlength)(inputx)
            #print(outputa,cin,encoder_outputsin,outputd_embed)

            #outputd=Lambda(lambda x:K.expand_dims(x,1))(outputd) #1, rnnsize




            return out


    embeddingFrTrain=EmbeddingFrTrain(len(target_vocab_to_int)  ,wordvectorsize)

In [31]:
print(type(embedding_layer))

<class 'tensorflow.python.keras.layers.embeddings.Embedding'>


In [32]:
def encoder_layer(rnn_inputs, rnn_size=200, rnn_num_layers=1,
                  source_sequence_len=maxlength):


    x=embedding_layer(rnn_inputs)

 
    
    rnn_sizenotfirst=int(rnn_size)
 
    for i in range(layerofencoderlstm):
        x, state_h, state_c,backward_h,backward_c=Bidirectional(LSTM(int(rnn_sizenotfirst),return_sequences=True,dropout=lstmdropout,return_state=True,recurrent_dropout=recur_dropout, kernel_regularizer=lstmregulizer, 
                                  recurrent_regularizer=lstmregulizer))(x)
    state_c = Concatenate()([state_c, backward_c])
    print('x',x)
    #x, state_h, state_c =LSTM(rnn_size,return_state=True,return_sequences=True,unroll=True) (x)
   

    #state_c=Dropout(betweenlayerdropout)(state_c)
    #state_c=Dense(rnn_size)(state_c)
    encoder_outputs=x
    encoder_states=state_c
    #print('encoder_states',encoder_states)
    #encoder_states= K.zeros_like(encoder_states)
 
    return encoder_outputs, encoder_states

In [33]:
def decoder_layer_inputs(target_data, target_vocab_to_int):

    withoutlast=target_data[:,:-1]
   # withoutlast = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])

    decoder_inputs = tf.concat([tf.fill([tf.shape(withoutlast)[0], 1], tf.cast(target_vocab_to_int["<GO>"],tf.float32)), 
                                withoutlast], 1)
    
    return decoder_inputs

In [34]:
if(attention=='Loung'):
    class Loung(tf.keras.layers.Layer):  #attention class
        def __init__(self, rnn_size,maxlength):
            super(Loung, self).__init__()
            self.W1 = Dense(wordvectorsize,activation='tanh')
            self.W2 = Dense(1,activation='relu')
            self.W3 = Dense(loungAttentionNodeCnt,activation=activationF,kernel_regularizer=denseregulizer,
                    activity_regularizer=denseregulizer)
            self.rnn_size=rnn_size
            self.maxlength=maxlength
            #self.lstm_cell = LSTM(rnn_size,return_state=True,dropout=decoderlstmdropout,recurrent_dropout=decoderrecurdropout,return_sequences=False,name='decoder_cell')
            self.lstm_cell = LSTM(rnn_size*2,return_state=True,
                                  dropout=decoderlstmdropout,
                                  recurrent_dropout=decoderrecurdropout,
                                return_sequences=False,
                                  kernel_regularizer=lstmregulizer, 
                                  recurrent_regularizer=lstmregulizer

                                 )

        def get_config(self):

            config = super().get_config().copy()
            config.update({

                'rnn_size':self.rnn_size,
                'maxlength':self.maxlength,

            })
            return config

        def call(self, outputa,cin,encoder_outputsin,outputd_embed,outputdlast):

            #print(outputa,cin,encoder_outputsin,outputd_embed)

            #outputd=Lambda(lambda x:K.expand_dims(x,1))(outputd) #1, rnnsize


            ###!!!!!!!!!!!!!!!!!!!!!!!!!!!#########
            zeros = Lambda(lambda x: tf.zeros_like(x))(cin)
            #print('outputa',outputa)

            outputa=tf.reshape(outputa,(-1,wordvectorsize))
            outputdlast=tf.reshape(outputdlast,(-1,loungAttentionNodeCnt))
            #print('Concatenate1inpuy',outputa,outputdlast)
            outputa=Concatenate(axis=-1)([outputa,outputdlast])
            outputa=tf.expand_dims(outputa, 1)


            #print('input',outputa,cin,encoder_outputsin,outputd_embed)
            outputa,_,  c = self.lstm_cell(outputa,initial_state= [zeros,cin] )  
           # print('lstmout',outputa,c)

            outputd=RepeatVector(maxlength)(outputa) #25, wordvectorsize
          #  print('RepeatVectorout',outputd)
            concate=Concatenate(axis=-1)([encoder_outputsin,outputd]) #25 2*rnnsize+2*rnnsize
           # print('Concatenateout',concate)
            concate=self.W1(concate) #25,100 
            #print('concate',concate)

            score=self.W2(concate) #25,1 ############score
            #score=tf.matmul(s2,outputd,transpose_a=True) #100,25     25,100
            #print('score',score)
            #100*100




            attention_weights = Softmax(axis=-2)(score) #25,1
            #print('attention_weights',attention_weights)
            attention_weights=K.squeeze(attention_weights,axis=-1)
        

            context_vector = Dot(axes=1)([attention_weights,encoder_outputsin]) #25,1   25 rnnsize ->#1,rnnsize


            #print('Concatenateout2',concate)
            outputd = tf.concat([tf.expand_dims(context_vector, 1), tf.expand_dims(outputa, 1)], axis=-1)  #1,rnnsize+(wordvectorsize)
            #print('outputdAfterconcat',outputd)
            #outputd = tf.concat([tf.expand_dims(context_vector, 1),outputsoft], axis=-1) 


            #print('outputd',outputd)
            outputd=self.W3(outputd)
            outputd=Dropout(betweenlayerdropout)(outputd)

            return outputd, c


    attention_decoder1=Loung(rnn_size,maxlength)

### 3.2.2 - Decoder Traing

In [35]:
def decoder_layer_the(encoder_outputs,encoder_states, is_training, decoder_embed,
                        target_sequence_len, max_target_sequence_len,rnn_size):
    

    
    
    outputsall=[]

    c=encoder_states
    outputd_embed=tf.zeros((tf.shape(encoder_states)[0],wordvectorsize))
    if(attention=='Loung'):
        
        outputd=tf.zeros((tf.shape(encoder_states)[0],loungAttentionNodeCnt))
    else:
        outputd=tf.zeros((tf.shape(encoder_states)[0],wordvectorsize))
   # print('c',c)

    #print(decoder_embed) #vectors of 100
    #attention_decoder=attention_decoder(rnn_size,target_sequence_len)
    

    for i in range(max_target_sequence_len):
        if(forceteach==1):
            if(is_training==1):
                
                random=K.random_uniform(
                    (1,),
                    minval=0.0,
                    maxval=5.0,

                )
                random=int(K.get_value(random))
                a=i+2-random #(i+1 - (random 0 to 2))
                if (a<0):
                    a=0
                outputd_embed=decoder_embed[:,a]  #go

         

            
            else:
                if(i==0):
                    outputd_embed=decoder_embed[:,i]  #go

        
                else:
                    outputd_embed=outputd_embed
        else: #forceteach==0
            outputd_embed=outputd_embed

            #outputd= Lambda(lambda x: K.in_train_phase(decoder_embed[:,i],outputd))(output)
        
        outputd,c=attention_decoder1(outputd_embed,c,encoder_outputs,outputd_embed,outputd)
     
        
        outputdsoftmax=Dense(len(target_vocab_to_int),activation='softmax')(outputd)
        outputd_embed=K.argmax(outputdsoftmax)
        print('outputdbeforeembedding',outputd_embed)
        if(pretrain_fr==1):
            
            outputd_embed=embedding_layerfr(outputd_embed)
        else:
            outputd_embed=embeddingFrTrain(outputd_embed)
      
        print(outputd_embed)
       
        outputsall.append(outputdsoftmax)

             

    #print('K.is_keras_tensor(outputsall)',K.is_keras_tensor(outputsall[0]))
    print('outputd',outputd)
    print('outputsall',outputsall)
    outputsall=Lambda(lambda x:K.stack(x,axis=1))(outputsall)
    print('outputsall',outputsall)
    #outputsall = Lambda(lambda x: K.squeeze(x,2))(outputsall)
   # outputsall=outputsall+outputd
    print('outputsall2',outputsall)
    return outputsall

In [36]:
def decoder_layer(encoder_outputs,encoder_states,is_training, decoder_inputs, 
                    rnn_size, rnn_num_layers,
                   target_vocab_to_int,
                  target_sequence_len=maxlength,
                  max_target_sequence_len=maxlength):

    
    if(pretrain_fr==1):
        
        decoder_embed=embedding_layerfr(decoder_inputs) #output as vector
    else:
        
        decoder_embed=embeddingFrTrain(decoder_inputs)
    

    
    # output_layer logits

    
    
    logits = decoder_layer_the(encoder_outputs,encoder_states,is_training,

                                           decoder_embed,
                                           target_sequence_len,
                                           max_target_sequence_len,
                                         rnn_size=rnn_size)
    

    
    return logits

In [37]:
def seq2seq_model(  
                 shape,source_sequence_len, target_sequence_len, max_target_sequence_len,
          
          
                 rnn_size, rnn_num_layers, target_vocab_to_int):
    
    input_data = Input(shape=shape)
    target_data= Input(shape=shape)
  


    encoder_outputs, encoder_states = encoder_layer(input_data, rnn_size, rnn_num_layers, source_sequence_len)
    

    decoder_inputs = decoder_layer_inputs(target_data, target_vocab_to_int)  
    
    is_training= K.in_train_phase(1,0)
    
    output = decoder_layer(encoder_outputs,encoder_states,is_training,
                                                                       decoder_inputs,
                                                                      target_sequence_len=target_sequence_len,
                                                                       max_target_sequence_len=max_target_sequence_len,
                                                                      rnn_size=rnn_size,
                                                                      rnn_num_layers=rnn_num_layers,
                                                                      target_vocab_to_int=target_vocab_to_int
                                                                     
                                                                   
                                                                      )
    


    
    model = Model([input_data,target_data], output)
    return model

In [38]:
# Number of Epochs

# Batch Size

# RNN Size
rnn_size = rnn_size
# Number of Layers
rnn_num_layers = 1


In [39]:

if(loadmodel==0):
    model = seq2seq_model(  (maxlength,),
                 maxlength, maxlength, maxlength,
               
                
                 rnn_size, rnn_num_layers, target_vocab_to_int)
   # model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
x Tensor("bidirectional/concat:0", shape=(?, 30, 800), dtype=float32)
outputdbeforeembedding Tensor("ArgMax:0", shape=(?, 1), dtype=int64)
Tensor("embedding_1_1/embedding_lookup/Identity_1:0", shape=(?, 1, 100), dtype=float32)
outputdbeforeembedding Tensor("ArgMax_1:0", shape=(?, 1), dtype=int64)
Tensor("embedding_1_2/embedding_lookup/Identity_1:0", shape=(?, 1, 100), dtype=float32)
outputdbeforeembedding Tensor("ArgMax_2:0", shape=(?, 1), dtype=int64)
Tensor("embedding_1_3/embedding_lookup/Identity_1:0", shape=(?, 1, 100), dtype=float32)
outputdbeforeembedding Tensor("ArgMax_3:0", shape=(?, 1), dtype=int64)
Tensor("embedding_1_4/embeddin

In [40]:
from tensorflow.keras.callbacks import *

checkpointer = ModelCheckpoint(filepath="./models/bestaccu.hdf5", 
                           monitor = 'val_loss',
                           mode='min',
                           verbose=1, 
                           save_best_only=True)   
tensorboard=TensorBoard(log_dir='./logs')

    

#model.compile(loss=customLoss, optimizer=adam) 
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=3,verbose=1,min_lr=0.000001)
earlystopping=EarlyStopping(monitor='val_loss', patience=50, verbose=2)


callbackss=[reduce_lr,earlystopping,
           #checkpointer,
            tensorboard]

In [41]:
#Lossf=tf.keras.losses.SparseCategoricalCrossentropy()

In [42]:
print(len(target_vocab_to_int))

5302


In [43]:
print(source_vocab_to_int.get("<UNK>"))
print(target_vocab_to_int.get("<UNK>"))

1
2


In [44]:
adam=tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001, amsgrad=True)
#adam=tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001)
#adam=tf.keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
#model.compile(loss=customloss, optimizer=adam)

In [45]:
print(Y.shape)
print(X.shape)


#n_values = np.max(Y) + 1
#Yhot=np.eye(n_values)[Y]
#print(Yhot.shape)

(233705, 30)
(233705, 30)


In [46]:
indexxx=2000
print(X[indexxx])
xlist=[]
for word in X[indexxx]:
    xlist.append(source_int_to_vocab[word])
xlist=' '.join(xlist)
print(xlist)

print(Y[indexxx])
ylist=[]
for word in Y[indexxx]:
    ylist.append(target_int_to_vocab[word])
ylist=' '.join(ylist)
print(ylist)

[2479 3138 2654 3608 1100  349 1525  401 2163 3897    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
i . financial performance report for the period from november <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
[3250 4110 5108  138 1867  190 3354  203 2402 2020    1    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
i . rapport sur l'exécution du budget pour la période <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [47]:
from sklearn.utils.class_weight import compute_sample_weight
class_weights =compute_sample_weight('balanced',y=Y)

In [48]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 30)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 30, 100)      401200      input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   [(None, 30, 800), (N 1603200     embedding[0][0]                  
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 800)          0           bidirectional[0][2]              
                                                                 bidirectional[0][4]          

In [49]:
#istraining=np.ones((X.shape[0]))



model.fit([X, Y], Y, epochs=500, validation_split=0.1,batch_size=60,
          callbacks = callbackss
          #, class_weight=class_weights
          #,shuffle=False
           )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 210334 samples, validate on 23371 samples
Epoch 1/500
210334/210334 [==============================] - 791s 4ms/sample - loss: 2.8785 - val_loss: 3.1965
Epoch 2/500
210334/210334 [==============================] - 802s 4ms/sample - loss: 2.5892 - val_loss: 3.0894
Epoch 3/500
210334/210334 [==============================] - 809s 4ms/sample - loss: 2.4924 - val_loss: 3.0294
Epoch 4/500
210334/210334 [==============================] - 814s 4ms/sample - loss: 2.4347 - val_loss: 2.9884
Epoch 5/500
210334/210334 [==============================] - 815s 4ms/sample - loss: 2.3949 - val_loss: 2.9577
Epoch 6/500
210334/210334 [==============================] - 812s 4ms/sample - loss: 2.3653 - val_loss: 2.9353
Epoch 7/500
210334/210334 [==============================] - 818s 4ms/sample - loss: 2.3418 - val_loss: 2.9194
Epoch 8/500
210334/210334 [==============================] - 814s 4ms/sample -

KeyboardInterrupt: 

In [ ]:
source_text.split("\n")[:51]

In [ ]:
istrainingx=np.asarray([1])

yy=np.expand_dims(Y[50], axis=0)
print(Y[50])

def make_prediction(sentence):

    unk_idx = source_vocab_to_int["<UNK>"]
    word_idx=[]
    for word in sentence.lower().split():
        

        word=source_vocab_to_int.get(word, unk_idx)
        word_idx.append(word)
    #word_idx = [source_vocab_to_int.get(word, unk_idx) for word in sentence.lower().split()]

    word_idx = np.array(word_idx + [0] * (maxlength - len(word_idx)))
    word_idx=word_idx[:maxlength]
   # print(word_idx)

    preds = model.predict([word_idx.reshape(-1,maxlength), yy, istrainingx])
    predictions = np.argmax(preds, axis=-1)
    
   


    idx = [target_int_to_vocab.get(int(idx), "<UNK>") for idx in predictions[0]]
 
 
    return " ".join(idx)

In [ ]:
print(source_int_to_vocab[1])

In [ ]:
#source_text.split("\n")[:100]

In [ ]:
number=8000
print(source_text.split("\n")[number])
make_prediction(source_text.split("\n")[number])

In [ ]:
make_prediction('government')

In [ ]:
translate_sentence_text = ""

In [ ]:
translate_sentence = sentence_to_seq(translate_sentence_text, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph('checkpoints/dev.meta')
    loader.restore(sess, tf.train.latest_checkpoint('./checkpoints'))

    input_data = loaded_graph.get_tensor_by_name('inputs:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_len:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_len:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size})[0]

print('【Input】')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\n【Prediction】')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format([target_int_to_vocab[i] for i in translate_logits]))

print("\n【Full Sentence】")
print(" ".join([target_int_to_vocab[i] for i in translate_logits]))

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

In [ ]:

fr_preds = []


for sentence in (source_text.split("\n")[:2000]):
    fr_pred = make_prediction(sentence)
   
    fr_preds.append(fr_pred)

In [ ]:

references = target_text.split("\n")[:2000]

In [ ]:

bleu_score = []

for i in tqdm.tqdm(range(len(fr_preds))):
 
    pred = fr_preds[i].replace("<EOS>", "").replace("<PAD>", "").rstrip()
    reference = references[i].lower()
   
    score = sentence_bleu([reference.split()], pred.split())
    if(score<0.8):
        print(pred)
        print(reference)
    bleu_score.append(score)

In [ ]:
print("The BLEU score is  {}".format(sum(bleu_score) / len(bleu_score)))